# Table Summary

Here is my understanding of the data structure.

* `orders`
    * one row per order (index = order_id)
    * does not contain information about reorders
    * `eval_set` indicates whether the order is in the `train`/`test`/`prior`
        * the `test` set is data reserved for the testing of our final model
        * the `prior` and `train` eval_sets are defined below
    * columns:
        * `order_id`: order identifier
        * `user_id`: customer identifier
        * `eval_set`: which evaluation set this order belongs in (see `SET` described below)
        * `order_number`: the order sequence number for this user (1 = first, n = nth)
        * `order_dow`: the day of the week the order was placed on
        * `order_hour_of_day`: the hour of the day the order was placed on
        * `days_since_prior`: days since the last order, capped at 30 (with NAs for `order_number` = 1)

* `prior_orders`
    * information about orders prior to that users most recent order (~3.2M orders)
    * contains one row per item per order & whether or not each item is a 'reorder'
        * reorder: 1 if products has been ordered by this user in the past, 0 otherwise
    * columns:
        * `order_id`: foreign key
        * `product_id`: foreign key
        * `add_to_cart_order`: order in which each product was added to cart
        * `reordered`: 1 if this product has been ordered by this user in the past, 0 otherwise
        
    
* `train_orders`
    * training data supplied to participants of Kaggle competition
    * this table represents the users' most recent orders
    * contains one row per item per order & whether or not each item is a 'reorder'(for training data)
    * none of the rows in `train_orders` will be found in `prior_orders`
    * columns:
        * `order_id`: foreign key
        * `product_id`: foreign key
        * `add_to_cart_order`: order in which each product was added to cart
        * `reordered`: 1 if this product has been ordered by this user in the past, 0 otherwise

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import psycopg2


from db_config import get_db_params
from query_dfs import create_dfs

pd.set_option("display.max_columns", 101)

In [2]:
db_params = get_db_params()
conn = psycopg2.connect(**db_params)

In [3]:
df_orders, df_train, df_prior, df_prod_detail = create_dfs()

In [82]:
# pickle all DFs for ease of use later
# comment this cell after executing once (provided no changes to DB/query_dfs)

# df_orders.to_pickle("./pickle/df_orders.pickle")
# df_train.to_pickle("./pickle/df_train.pickle")
# df_prior.to_pickle("./pickle/df_prior.pickle")
# df_prod_detail.to_pickle("./pickle/df_prod_detail.pickle")


**Let's take a look at how our DataFrames are structured.**

In [4]:
df_orders.shape, df_train.shape, df_prior.shape, df_prod_detail.shape

((3421083, 7), (1384617, 10), (32434489, 10), (49688, 5))

In [5]:
df_orders.head(3)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0


In [6]:
df_train.head(3)

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,6129,24852,1,1,38907,train,7,1,14,30.0
1,6129,48364,2,1,38907,train,7,1,14,30.0
2,6129,21903,3,1,38907,train,7,1,14,30.0


In [7]:
df_prior.head(3)

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,114,24954,1,0,91891,prior,1,0,11,NaN
1,114,1688,2,0,91891,prior,1,0,11,NaN
2,114,37371,3,0,91891,prior,1,0,11,NaN


In [8]:
df_prod_detail.head(3)

,aisle_id,department_id,product_name,aisle,department
product_id,,,,,
30843,1,20,Detox Salad,prepared soups salads,deli
16618,1,20,Classic Potato Salad,prepared soups salads,deli
14864,1,20,Low-Fat Chicken Tortilla Soup,prepared soups salads,deli


In [9]:
user_prod_counts = (df_prior
                    .groupby(["product_id", "user_id"], as_index=False)
                    .agg({"order_id": "count"})
                    .rename(columns={'order_id': 'num_orders'}))

user_prod_counts.head(3)

,product_id,user_id,num_orders
0,1,138,2
1,1,709,1
2,1,764,2


## Feature Engineering

**We will use the `df_orders`, `df_train`, `df_prior`, and `df_prod_detail` DataFrames to populate a new DataFrame, `X`.**

**`X` will contain all of the features we'll use for our modeling.**

In [57]:
train_user_ids = df_train['user_id'].unique() 
X = user_prod_counts[user_prod_counts['user_id'].isin(train_user_ids)]
X.head(3)

,product_id,user_id,num_orders
0,1,138,2
1,1,709,1
3,1,777,1


In [58]:
train_user_ids

array([38907, 87425, 64764, ..., 84438, 36854,  9808])

In [59]:
X.head(3)

,product_id,user_id,num_orders
0,1,138,2
1,1,709,1
3,1,777,1


In [60]:
train_carts = (df_train.groupby('user_id', as_index=False)
                                      .agg({'product_id': 'unique'})
                                      .rename(columns={'product_id': 'cart'}))
train_carts.head(3)

,user_id,cart
0,1,"[196, 25133, 38928, 26405, 39657, 10258, 13032..."
1,2,"[22963, 7963, 16589, 32792, 41787, 22825, 1364..."
2,5,"[15349, 19057, 16185, 21413, 20843, 20114, 482..."


In [61]:
# DO NOT RE-RUN THIS CELL (if you do, must re-run all from where X is instantiated to fix)
X = X.merge(train_carts, on="user_id")
X.head(3)

,product_id,user_id,num_orders,cart
0,1,138,2,[42475]
1,907,138,2,[42475]
2,1000,138,1,[42475]


**CAUTION**: the below cell takes a couple of minutes to run. To account for this, I pickle it immediately afterwards. Once you've pickled it once, comment out this line (two cells down):

`X.to_pickle(X_5_pickle_path)`

This will prevent the need to re-run this expensive operation.

In [62]:
X['in_cart'] = (X.apply(lambda row: row['product_id'] in row['cart'], axis=1).astype(int))

X.head(3)

,product_id,user_id,num_orders,cart,in_cart
0,1,138,2,[42475],0
1,907,138,2,[42475],0
2,1000,138,1,[42475],0


In [63]:
# save this DF to pickle (X_5 since we have 5 features @ this checkpoint)
X_5_pickle_path = "./pickle/X_5.pickle"
# X.to_pickle(X_5_pickle_path)

Now that we've done this, let's move to `feature_engineering_2`, where we will pick up where we left off by reading this pickled file.